In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.stats.api as sms
from sklearn.model_selection import train_test_split

# Načítavanie dát

In [2]:
data = pd.read_csv('data/data_phase_1.csv')

# Podiel práce

Tulach 50%

Černega 50%

# Riešenie

## 2.1 A

Na konci prvej fázy zadania sme si exportli mergnutý dataset všetkých 4 tabuliek a ten sme následne importovali sem:

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45586 entries, 0 to 45585
Data columns (total 49 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   ts                          45586 non-null  object 
 1   imei                        45586 non-null  int64  
 2   mwra                        45586 non-null  float64
 3   c.android.youtube           45586 non-null  float64
 4   c.android.chrome            45586 non-null  float64
 5   c.android.gm                45586 non-null  float64
 6   c.dogalize                  45586 non-null  float64
 7   c.katana                    45586 non-null  float64
 8   c.UCMobile.x86              45586 non-null  float64
 9   c.updateassist              45586 non-null  float64
 10  c.android.vending           45586 non-null  float64
 11  c.UCMobile.intl             45586 non-null  float64
 12  c.raider                    45586 non-null  float64
 13  p.android.packageinstaller  455

Moje zdôvodnenie pre tento krok je ten, že sme v prvej fáze niektoré stĺpce niektorých tabuliek mazali a prerábali na dva osobitné stĺpce (konkrétne stĺpec location sme úplne zmazali a tie dáta sme prerobili na latitude a longtitude, keďže je lepšie mať univerzálny formát dát). Sekvenčne je vytvorenie testovacej a trénovacej sady z týchto dát ako prvá úloha v tejto fáze zadania, a síce už máme odstránených outlierov, mohli nám vzniknúť noví, ktorých by sme dokázali odstrániť pomocou Pipeline tak či tak. Vytvoríme si teda testovaciu a trénovaciu sadu:

In [4]:
# Define the target variable and features
X = data.drop(columns=['mwra'])
y = data['mwra']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Display the shapes of the resulting datasets
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

X_train shape: (36468, 48)
X_test shape: (9118, 48)
y_train shape: (36468,)
y_test shape: (9118,)


Ďalej budeme už robiť zatiaľ s trénovacím datasetom.

## 2.1 B